<a href="https://colab.research.google.com/github/rogeriomoreirajr/TCC-PUC-Minas/blob/master/2_Coleta_e_Tratamento_Inicial_dos_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dados

Este notebook contém os scripts usados para capturar, tratar e salvar os dados usados no TCC

## Parâmetros
- Todas as datas salvas como datetime
- Preços como floats

## 2.1. Preço do combustível

> - Dados semanais (baseados no Excel, os municipais, estaduais e nacionais)
- Dados dos postos (conseguidos pelo preco.anp)

### Dados semanais
Semanalmente a ANP promove uma pesquisa nos postos de combustível para acompanhar os preços praticados. Um dos objetivos é poder checar se os postos não estão com alguma prática que possa prejudicar o consumidor (como cartéis). Como é de de imaginar (dado o tamanho continental do Brasil) essa pesquisa é feita por amostragem, em 459 municípios dos 27 estados e do Distrito Federal.

Os dados são publicados no site da autarquia de duas formas:
- Séries históricas: tabelas onde cada linha representa o resultado da pesquisa semanal para cada abrangência (municipal, estadual, regional ou nacional). São tabelas de excel, com os valores mais antigos datando de 2004. Cada linha traz dados como o preço máximo, médio e mínimo de revenda, além do desvio padrão (entre outros).
- Levantamento de preço: uma interface que permite buscar os resultados da última coleta, por município e estado. Através de requisições diretas ao servidor é possível ter os dados das últimas dez coletas.

#### Dados de município

In [0]:
import pandas as pd
import os
from datetime import datetime, timedelta

# Essa biblioteca abre os arquivos xlsb, um arquivo binário do Excel
!pip install pyxlsb
from pyxlsb import open_workbook as open_xlsb

# As tabelas estão disponíveis em http://www.anp.gov.br/precos-e-defesa-da-concorrencia/precos/levantamento-de-precos/serie-historica-do-levantamento-de-precos-e-de-margens-de-comercializacao-de-combustiveis
# O que fiz aqui foi salvá-las localmente (no caso, no Google Drive, montado no
# Google Colab) e lê-las a partir daí, como arquivos locais.
path = '/content/drive/My Drive/EAD/PUC Minas/13 - TCC/data/raw/anp'
os.chdir(path)
files = os.listdir()

# Separo em duas listas: uma com os xlsb e outra com xlsx
# Isso é necessário porque o xlsb não é suportado pelo pandas,
# e precisa de outro tratamento.
files_xlsb = [el for el in files if el.endswith('xlsb')]
files_xlsx = [el for el in files if el.endswith('xlsx')]

list_dfs = []

# A função abaixo abre o xlsb, desde que a gente dê a linha do cabeçalho

def xlsb2pandas(file, header):
    """
    file: caminho para o arquivo
    header: linha onde está o cabeçalho
    """
    values = []

    with open_xlsb(file) as wb:
        with wb.get_sheet(1) as sheet:
            for row in sheet.rows():
                values.append([item.v for item in row])

    df_ = pd.DataFrame(values[header+1:])

    # Transformar a data de cinco dígitos em datetime
    df_[0] = df_[0].apply(lambda x:(datetime(1899, 12, 30) + timedelta(days= x )))
    df_[1] = df_[1].apply(lambda x:(datetime(1899, 12, 30) + timedelta(days= x )))

    headers = values[header]
    df_.columns = headers

    return df_


df_ = xlsb2pandas(files_xlsb[0], 12)
list_dfs.append(df_)

df_ = xlsb2pandas(files_xlsb[1], 14)
list_dfs.append(df_)


# Dados recentes
for file in files_xlsx:
    skip = 12
    if file  == 'SEMANAL_MUNICIPIOS-2018.xlsx':
        # Tem uma observação no arquivo de 2018, logo uma linha a mais
        skip = 13
    
    df_ = pd.read_excel(file, skiprows=	skip)
    list_dfs.append(df_)

# Um df com todos os dados
df_mun = pd.concat(list_dfs, ignore_index=True)

df_mun.columns = [
 'data_inicial',
 'data_final',
 'regiao',
 'estado',
 'municipio',
 'produto',
 'número_postos_pesquisados',
 'unidade_medida',
 'preço_medio_revenda',
 'desvio_padrao_revenda',
 'preço_minimo_revenda',
 'preço_máximo_revenda',
 'margem_media_revenda',
 'coef_variaçao_revenda',
 'preço_medio_distribuiçao',
 'desvio_padrao_distribuiçao',
 'preço_minimo_distribuiçao',
 'preço_máximo_distribuiçao',
 'coef_variaçao_distribuiçao']

select = ['data_inicial',
 'data_final',
 'regiao',
 'estado',
 'municipio',
 'preço_medio_revenda',
 'preço_minimo_revenda',
 'preço_máximo_revenda',]

df_mun = df_mun[df_mun.produto == 'GASOLINA COMUM'][select]

# Algumas datas estão em formato numérico

path = '/content/drive/My Drive/EAD/PUC Minas/13 - TCC/data/'
df_mun.to_csv(path+'/anp_municipios.csv', index=False)

#### Dados nacionais, regionais e estaduais

In [0]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

url = 'http://www.anp.gov.br/precos-e-defesa-da-concorrencia/precos/levantamento-de-precos/serie-historica-do-levantamento-de-precos-e-de-margens-de-comercializacao-de-combustiveis'
root = 'http://www.anp.gov.br/'

soup = BeautifulSoup(requests.get(url).content)

def not_city(link):
    href = link['href']
    if href.endswith('xlsx'):
        if 'MUNICIPIO' not in href:
            if 'SEMANAL' in href or 'Semanal' in href:
                return True   
        
links = [el['href'] for el in soup.findAll('a') if not_city(el)]

list_dfs = []

for file in links:
    df_ = pd.read_excel(root+file)

    # achar o cabeçalho
    ix = df_[df_.iloc[0:,0] == 'DATA INICIAL'].index[0]
    df_ = pd.read_excel(root+file, skiprows=ix, header=1)
    
    list_dfs.append(df_)

df = pd.concat(list_dfs, ignore_index=True, sort=False)


def tipo_dado(row):
    """
    Essa função analisa se a linha pertence a um dado nacional, estadual ou regional
    """
    if pd.isnull(row['REGIÃO']) and  pd.isnull(row['ESTADO']):
        return 'nacional'
    elif not pd.isnull(row['REGIÃO']) and pd.isnull(row['ESTADO']):
        return 'regional'
    elif not pd.isnull(row['REGIÃO']) and not pd.isnull(row['ESTADO']):
        return 'estadual'

df['abrangencia'] = df.apply(tipo_dado, axis=1)  

df.columns = ['data_inicial',
 'data_final',
 'produto',
 'número_de_postos_pesquisados',
 'unidade_de_medida',
 'preco_medio_revenda',
 'desvio_padrao_revenda',
 'preco_minimo_revenda',
 'preco_maximo_revenda',
 'margem_media_revenda',
 'coef_de_variacao_revenda',
 'preco_medio_distribuicao',
 'desvio_padrao_distribuicao',
 'preco_minimo_distribuicao',
 'preco_maximo_distribuicao',
 'coef_de_variacao_distribuicao',
 'regiao',
 'estado',
 'abrangencia']

not_include = [ 'margem_media_revenda',
 'produto',
 'número_de_postos_pesquisados',
 'unidade_de_medida',
 'coef_de_variacao_revenda',
 'preco_medio_distribuicao',
 'desvio_padrao_distribuicao',
 'preco_minimo_distribuicao',
 'preco_maximo_distribuicao',
 'coef_de_variacao_distribuicao',
]

columns = [el for el in df.columns if el not in not_include]

df_abrangencia = df[df.produto == 'GASOLINA COMUM'][columns]

for data in ['data_final','data_inicial']:
    df[data] = pd.to_datetime(df_abrangencia[data])

path = '/content/drive/My Drive/EAD/PUC Minas/13 - TCC/data/'
df_abrangencia.to_csv(path+'anp_abrangencias.csv', index=False)

### Dados dos postos
A ANP divulga no site http://preco.anp.gov.br/ o resultado da última coleta em cada município, através do sistema de busca. Para conseguir os dados de outras semanas (as últimas dez, que são as que estão disponíveis) eu fiz um script que faz o scrape no site, mandando uma requisição POST e tratando os dados que o site retorna.

In [0]:
"""
Ele não tem dados de muitas semanas atrás?
"""

import pandas as pd
import re
import os

path = '/content/drive/My Drive/EAD/PUC Minas/13 - TCC/data/raw'
os.chdir(path)

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
headers = {'User-Agent': user_agent}

import requests
from bs4 import BeautifulSoup
from time import sleep

url = 'http://preco.anp.gov.br/include/Relatorio_Excel_Semanal_Posto.asp'

# Cidades ao redor de Belo Horizonte
cidades = {
    2762: "Betim",
    2980: "Contagem",
    2754: "Belo Horizonte",
    3724: "Sabará",
    3480: "Nova Lima",
}

# Dados universais sobre os combustiveis
combustiveis = {
    '487':'gasolina',
    '532':'diesel', 
    }

data = {
    'btnSalvar':'Exportar',
    'COD_SEMANA': None,
    'COD_COMBUSTIVEL': None, 
    'COD_MUNICIPIO': None, 
    }

# Valor da semana atual
url = 'http://preco.anp.gov.br/include/Resumo_Por_Municipio_Index.asp'
soup_ = BeautifulSoup(requests.get(url).content)
end = soup_.find('form',{'id':'frmAberto'}).find('input',{'name':'cod_Semana'})['value']


url = 'http://preco.anp.gov.br/include/Relatorio_Excel_Semanal_Posto.asp'

# Último valor lido
file_ = '/content/drive/My Drive/EAD/PUC Minas/13 - TCC/data/preços nos postos.csv'
start = pd.read_csv(file_).ix_semana.max()

semanas = range(start+1, int(end)+1)

l_precos = []

for cidade in cidades:
    data['COD_MUNICIPIO'] = cidade
    
    n_cidade = cidades[cidade]
    print(n_cidade)

    for combustivel in combustiveis:
        data['COD_COMBUSTIVEL'] = combustivel
        
        n_combustivel = combustiveis[combustivel]
        print('>>>'+n_combustivel, end='\t')

        for semana in semanas:
            print('.', end='')

            data['COD_SEMANA'] = semana
            content = requests.post(
                url, 
                data = data,
                headers=headers).content

            soup = BeautifulSoup(content)

            tabelas = pd.read_html(str(soup))

            def loc_df(df):
                df['combustivel'] = n_combustivel
                df['cidade'] = n_cidade
                df['ix_semana'] = semana

            df_precos = tabelas[1]
            loc_df(df_precos)
            l_precos.append(df_precos)

        print(' fim')


df = pd.concat(l_precos)

def one_level(tuple):
    """
    # A tabela que vem do site vem com dois níveis de colunas. 
    Essa função deixa em um nível
    """
    if tuple[1] == '': return tuple[0]
    else: return tuple[1]

df.columns = [one_level(el) for el in df.columns]

df.rename( columns=
{'BAIRRO': 'bairro',
 'BANDEIRA': 'bandeira',
 'DATA COLETA': 'data',
 'ENDEREÇO': 'endereco',
 'FORNECEDOR (B BRANCA)': 'fornecedor',
 'MODELIDADE DE COMPRA': 'modalidade',
 'PREÇO COMPRA': 'compra',
 'PREÇO VENDA': 'venda',
 'RAZÃO SOCIAL': 'razao',
 'cidade': 'cidade',
 'combustivel': 'combustivel',
 'ix_semana': 'ix_semana'}, inplace=True)

df.venda = df.venda.astype(int)/1000

df_old = pd.read_csv(file_)
df_old = df_old.iloc[:,1:]

df_postos = pd.concat([df, df_old])

df_postos.to_csv('anp_preços nos postos.csv', index=False)

NameError: ignored

## 2.2. Preço do barril de Petróleo Brent

Preço do barril do patróleto Brent, recuperados direto do site da Nasdaq. Valores a partir de 2007.

ler: https://www.nasdaq.com/articles/how-do-crude-oil-prices-affect-oil-stocks-2016-08-14


In [0]:
import pandas as pd

path = '/content/drive/My Drive/EAD/PUC Minas/13 - TCC/data/'

url = 'https://www.nasdaq.com/api/v1/historical/BZ:NMX/commodities/2004-01-01/2020-03-30'

df_barril = pd.read_csv(url)[['Date',' Close/Last']]
df_barril.columns = ['data','close']

df_barril.data = pd.to_datetime(df_barril.data)

df_barril.to_csv(path+'nasdaq_preco barril.csv', index=False)

df_barril

,data,close
0,2020-03-27,24.93
1,2020-03-26,26.34
2,2020-03-25,27.39
3,2020-03-24,27.15
4,2020-03-23,27.03
...,...,...
3084,2007-11-23,95.70
3085,2007-11-21,94.95
3086,2007-11-20,95.43
3087,2007-11-19,92.37


## 2.3. Câmbio do dólar
Estou usando os dados do Banco Central do Brasil, através do Ipeadata (base de dados públicos gerenciada pelo Instituto de Pesquisa Econômica Aplicada, fundação pública federal vinculada ao Ministério da Economia. No caso, estou usando a média do dia da taxa de câmbio real/dólar comercial (valor de compra). 

In [0]:
import pandas as pd

url = 'http://ipeadata.gov.br/ExibeSerie.aspx?serid=38590&module=M'
df_dolar = pd.read_html(url, skiprows=1)[0]
df_dolar.columns = ['data','cambio']

# data como datetime
df_dolar.data = pd.to_datetime(df_dolar.data, format='%d/%m/%Y')

# Só queremos os dados a partir de 2004
df_dolar = df_dolar[df_dolar.data>'2004-01-01']

# cambio de str para float
df_dolar.cambio = df_dolar.cambio.astype(int) / 10000

path = '/content/drive/My Drive/EAD/PUC Minas/13 - TCC/data'

df_dolar.to_csv(path + '/dolar.csv', index=False)

## 2.4. Cesta básica

Os dados vem do DIEESE. Ele permite que sejam baixados num xls. Tenho, no arquivo baixado, os dados a partir de 2004

In [0]:
import pandas as pd

path = '/content/drive/My Drive/EAD/PUC Minas/13 - TCC/data/raw/cesta basica dieese.xls'

df_cesta = pd.read_excel(path, skiprows=1, skipfooter=2)
df_cesta = df_cesta.melt(id_vars = 'Unnamed: 0', 
                        var_name = 'abrangencia', 
                        value_name='valor')

df_cesta.columns = ['data', 'abrangencia', 'valor']

df_cesta.data = df_cesta.data.str.extract('(\d+-\d+)')
df_cesta.data = pd.to_datetime(df_cesta.data, format='%m-%Y')

path = '/content/drive/My Drive/EAD/PUC Minas/13 - TCC/data'
df_cesta.to_csv(path + '/cesta basica.csv', index=False)

## 2.5. Inflação IPCA

O combustível é um dos subitems do IPCA. O IBGE tem os dados desses subitens de mês em mês, no site deles (e num servidor ftp). É importante ter o script que busca os dados no servidor do IBGE e os salva no nosso servidor para facilitar que outras pessoas possam refazer os meus passos.

In [0]:
from zipfile import ZipFile
from io import StringIO, BytesIO
import os
import pandas as pd
import re

"""
A parte de baixo desse código é para fazer tudo o que eu fiz, mas via ftp.
Ainda não está implementada. Usei os arquivos locais.
"""
# !pip install ftputil
# import ftputil

# host = ftputil.FTPHost('ftp.ibge.gov.br','anonymous','')
# path = 'Precos_Indices_de_Precos_ao_Consumidor/IPCA/Resultados_por_Subitem'
# host.chdir(path)
# for folder in [el for el in host.listdir('') if host.path.isdir(el)]:
#     print(folder)
#     for file in host.listdir(folder):
#         print('\t'+file)
#         host.download(folder+'/'+file, 'ipca/{}/{}/'.format(folder, file))

p = '/content/drive/My Drive/EAD/PUC Minas/13 - TCC/data/raw/ipca/'

os.chdir(p)
folders = [el for el in os.listdir() if os.path.isdir(el)]

lista_abas = {}

for folder in folders:
    print(folder)

    # Para cada pasta no diretório, ele caminha e alimenta o `lista abas` 
    # com o conteúdo das planilhas
    os.chdir(folder)
    files = os.listdir()
    for file_ in files:
        with ZipFile(file_) as myzip:
            excel_file = myzip.namelist()[0]
            with myzip.open(excel_file) as myfile:
                print('...'+excel_file)

                # Essa foi a forma de abrir o arquivo:
                # transformar ele num objeto ExcelFile
                df = pd.ExcelFile(BytesIO(myfile.read()))
                df = df.parse(sheet_name=None)

                lista_abas[excel_file] = df
    
    # Ele volta para o diretório raiz
    os.chdir('..')

list_ipca = []

indices = ('Índice Geral',
           'ÍNDICE GERAL',
           'INDICE GERAL',
           ' ÍNDICE GERAL')

# Como o Google Colab não tem um locale, tive que criar esse dicionário
# Para transformar as datas em datetime
meses = {'Abril': 'April',
 'Agosto': 'August',
 'Dezembro': 'December',
 'Fevereiro': 'February',
 'Janeirro': 'January',
 'Janeiro': 'January',
 'Julho': 'July',
 'Junho': 'June',
 'Maio': 'May',
 'Março': 'March',
 'Novembro': 'November',
 'Outubro': 'October',
 'Setembro': 'September'}

def limpa_periodo(string):
    """
    Função para transformar a data do período em datetime
    """
    string = re.sub('[- ]*IPCA[^ ]*[- ]*','', string)
    string = re.sub('(^ | $)','', string)
    string = string.title()
    # string = string.replace(meses)
    for mes, month in meses.items():
        string = string.replace(mes, month)

    string = re.sub(' De ', ' ', string)

    dt_string = pd.to_datetime(string, format='%B %Y')

    # transformar em datetime
    return dt_string

# Preciso tirar uma aba específica: 'RELATÓRIO (FÓRMULA E PREENCHER '

for planilha in lista_abas:
    for aba in lista_abas[planilha]:
        if 'RELATÓRIO' in aba or 'MENSAL SUBITEM IPCA' in aba:
            df = lista_abas[planilha][aba].dropna(how='all').reset_index(drop=True)
            primeiro_index = df[df.iloc[:,0].isin(indices)].index
            if not primeiro_index.empty:
                primeira_linha, = primeiro_index.values
                ix_headers = primeira_linha-1
                headers = df.iloc[ix_headers]
                headers[0] = 'categoria'
                df.columns = headers.values

                # ll.append(df.iloc[ix_headers-1,0])
                periodo = df.iloc[ix_headers-1,0]

                # tirar os valores sobre o cabeçalho
                df = df.iloc[ix_headers+1:]
                df['periodo'] = limpa_periodo(periodo)

                # tirar colunas nulas
                not_null = [el for el in df.columns if pd.notna(el)]

                list_ipca.append(df[not_null])

df = pd.concat(list_ipca)

# Série temporal da gasolina em BH
f_gas = df.categoria.str.lower().isin(['gasolina', 'gasolina ', ' gasolina'])
tb_gas = df[f_gas].set_index('periodo')[['BH','NACIONAL']]
tb_gas['item'] = 'gasolina'

# Série temporal do indice geral do IPCA em BH
f_ger = df.categoria.str.lower().isin([el.lower() for el in indices])
tb_ger = df[f_ger].set_index('periodo')[['BH','NACIONAL']]
tb_ger['item'] = 'indice geral'

df_ipca = pd.concat([tb_ger, tb_gas])

df_ipca = df_ipca.reset_index()
df_ipca = df_ipca.melt(
                id_vars=['periodo','item'], 
                value_vars=['BH','NACIONAL'],
                var_name='abrangencia',
                value_name = 'indice'
                )

# Montar uma tabela com a estrutura 
# periodo | abrangencia | gasolina | indice geral
df_ipca = df_ipca.pivot_table('indice', 
                              ['periodo', 'abrangencia'], 
                              'item', 
                              'sum')

df_ipca.index.names = ['data','abrangencia']

path = '/content/drive/My Drive/EAD/PUC Minas/13 - TCC/data'
df_ipca.to_csv(path + '/ipca.csv', index=True)

In [0]:
import pandas as pd

path = '/content/drive/My Drive/EAD/PUC Minas/13 - TCC/data'
df_ipca = pd.read_csv(path + '/ipca.csv', 
                      parse_dates=['periodo'],
                      index_col = [0,1])

def acumulado(Series):
    a = 1
    for v in Series.dropna().values:
        a = a + a*float(v)/100
    return round((a-1)*100, 2)

ano_a_ano = []
f_nacional = df_ipca.reset_index()['abrangencia'] == 'NACIONAL'

for ano in range(2006,2021):
    f_date = df_ipca.reset_index()['periodo'].dt.year == ano

    ff = f_nacional & f_date

    s_ = df_ipca[ff.values].agg(acumulado)
    dif = (s_.gasolina/s_['indice geral'])
    ano_a_ano.append(
        {'ano':ano,
         'gasolina': s_.gasolina,
         'geral': s_['indice geral'],
         'diff':dif}
    )

pd.DataFrame(ano_a_ano)

,ano,gasolina,geral,diff
0,2006,2.49,3.14,0.792994
1,2007,-0.68,4.46,-0.152466
2,2008,-0.26,5.90,-0.044068
3,2009,2.06,4.31,0.477958
4,2010,1.67,5.91,0.282572
5,2011,6.93,6.50,1.066154
6,2012,-0.39,5.84,-0.066781
7,2013,6.51,5.91,1.101523
8,2014,2.89,6.41,0.450858
9,2015,20.10,10.67,1.883786


## Dados das cidades


Podemos usar os dados do Atlas Brasil. Como o site proíbe o o scrape por robôs, estou usando uma cópia da planilha oferecida, acessada em 8/3/2020

In [0]:
import pandas as pd
import os

path = '/content/drive/My Drive/EAD/PUC Minas/13 - TCC/data/raw'

os.chdir(path)

list_dfs = pd.read_excel('atlas2013_dadosbrutos_pt.xlsx', sheet_name=None)

df_mun = list_dfs['MUN 91-00-10']
df_est = list_dfs['UF 91-00-10']
df_bra = list_dfs['BR 91-00-10']

# Escrevendo csv
df_mun.to_csv('geo_municipios.csv', index=False)
df_est.to_csv('geo_estados.csv', index=False)
df_bra.to_csv('geo_brasil.csv', index=False)